In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [2]:
path_train = "D:\\BSC IT DOCX\\Research Papers\\Research Internship-Columbia\\Indoor localization\\Val_dataset\\validationData.csv"

In [3]:
train_df = pd.read_csv(path_train,header = 0)

In [4]:
train_df.shape

(1111, 529)

In [13]:
train_df = train_df[:19930]
train_df.head(5)

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7515.916799,4.864890e+06,1,1,0,0,0,0,1380872703
1,100,100,100,100,100,100,100,100,100,100,...,100,-7383.867221,4.864840e+06,4,2,0,0,0,13,1381155054
2,100,100,100,100,100,100,100,100,100,100,...,100,-7374.302080,4.864847e+06,4,2,0,0,0,13,1381155095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7365.824883,4.864843e+06,4,2,0,0,0,13,1381155138
4,100,100,100,100,100,100,100,100,100,100,...,100,-7641.499303,4.864922e+06,2,0,0,0,0,2,1380877774


In [18]:
train_AP_strengths = train_df.iloc[:,:520]

In [19]:
train_AP_strengths

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
3,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1107,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1108,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1109,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


In [20]:
#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths))

C:\Users\Anuj\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [21]:
#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

In [23]:
res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

In [28]:
train_labels.shape

(1111,)

In [25]:
res

0       11
1       24
2       24
3       24
4       02
        ..
1106    23
1107    23
1108    00
1109    00
1110    00
Length: 1111, dtype: object

In [27]:
#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)
dummy_labels

,00,01,02,03,10,11,12,13,20,21,22,23,24
0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,0,0,0,0,0,0,0,0,0,0,1,0
1107,0,0,0,0,0,0,0,0,0,0,0,1,0
1108,1,0,0,0,0,0,0,0,0,0,0,0,0
1109,1,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13.
train_labels.shape

(1111, 13)

In [30]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

In [32]:
train_val_split.shape

(1111,)

In [33]:
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [34]:
train_X.shape

(784, 520)

In [35]:
val_X.shape

(327, 520)

In [36]:
nb_epochs = 5
batch_size = 10
input_size = 520
num_classes = 13

In [37]:
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dense(64, activation='tanh', bias=True))
    return model

In [38]:
def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    e.add(Dense(256, activation='tanh', bias=True))
    e.add(Dense(input_size, activation='tanh', bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e

In [39]:
e = encoder()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, input_dim=520, activation="tanh", use_bias=True)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="tanh", use_bias=True)`
  after removing the cwd from sys.path.
C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="tanh", use_bias=True)`
  """


In [40]:
d = decoder(e)

C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=64, activation="tanh", use_bias=True)`
  
C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="tanh", use_bias=True)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(520, activation="tanh", use_bias=True)`
  after removing the cwd from sys.path.


In [41]:
d.fit(train_X, train_X, nb_epoch=nb_epochs, batch_size=batch_size)

C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.



Epoch 1/5
784/784 [==============================] - 3s 4ms/step - loss: 0.6032
Epoch 2/5
784/784 [==============================] - 1s 1ms/step - loss: 0.5425
Epoch 3/5
784/784 [==============================] - 1s 824us/step - loss: 0.5267
Epoch 4/5
784/784 [==============================] - 1s 890us/step - loss: 0.5176
Epoch 5/5
784/784 [==============================] - 1s 708us/step - loss: 0.5120


In [42]:

def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    d.add(Dense(128, activation='tanh', bias=True))
    d.add(Dense(num_classes, activation='softmax', bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return d


In [43]:
c = classifier(d)

C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=64, activation="tanh", use_bias=True)`
  """
C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="tanh", use_bias=True)`
  
C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(13, activation="softmax", use_bias=True)`
  import sys


In [44]:
c.fit(train_X, train_y, validation_data=(val_X, val_y), nb_epoch=nb_epochs, batch_size=batch_size)

C:\Users\Anuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 784 samples, validate on 327 samples
Epoch 1/5
784/784 [==============================] - 2s 3ms/step - loss: 1.0172 - accuracy: 0.6480 - val_loss: 0.7625 - val_accuracy: 0.7034
Epoch 2/5
784/784 [==============================] - 1s 2ms/step - loss: 0.4228 - accuracy: 0.8469 - val_loss: 0.6534 - val_accuracy: 0.7645
Epoch 3/5
784/784 [==============================] - 1s 967us/step - loss: 0.2821 - accuracy: 0.9056 - val_loss: 0.6111 - val_accuracy: 0.7859
Epoch 4/5
784/784 [==============================] - 1s 853us/step - loss: 0.1683 - accuracy: 0.9401 - val_loss: 0.6381 - val_accuracy: 0.7890
Epoch 5/5
784/784 [==============================] - 1s 667us/step - loss: 0.1740 - accuracy: 0.9388 - val_loss: 0.7017 - val_accuracy: 0.7920


In [47]:
loss, acc = c.evaluate(train_AP_features, train_labels)

1111/1111 [==============================] - 0s 266us/step


In [48]:
print( loss, acc)

0.28745290591414185 0.917191743850708
